In [1]:
# Import packages.
import cvxpy as cp
import numpy as np
import scipy 
import scipy.sparse as sparse

np.random.seed(0)

In [4]:
# Generate a random SDP.
n = 3
p = 4
np.random.seed(1)
C = np.random.randn(n, n)
A = [np.eye(n)]
b = [np.random.randn(n)]
c = [np.random.random()]
for i in range(p):
    A.append(np.random.randn(n, n))
    b.append(np.random.randn(n))
    c.append(np.random.random())

In [8]:
# Define and solve the CVXPY problem.
# Create a symmetric matrix variable.
X = cp.Variable((n,n), symmetric=True)
x = cp.Variable((n,))
# The operator >> denotes matrix inequality.
constraints = [X - cp.outer(x, x.T) >> 0]
constraints += [
    cp.trace(A[i] @ X) + b[i].T@x == c[i] for i in range(p)
]
prob = cp.Problem(cp.Minimize(cp.trace(C @ X)),
                  constraints)
prob.solve(solver = cp.SCS)

# Print result.
print("The optimal value is", prob.value)
print("A solution X is")
print(X.value)

DCPError: Problem does not follow DCP rules. Specifically:
The following constraints are not DCP:
var223 + -reshape(var224, (3, 1), F) @ reshape(var224, (1, 3), F) + Promote(-0.0, (3, 3)) >> 0 , because the following subexpressions are not:
|--  reshape(var224, (3, 1), F) @ reshape(var224, (1, 3), F)

In [ ]:
X = cp.Variable

In [37]:
new_A = constraints[1].dual_value*A[0]
# Weight b also!
X = cp.Variable((n,n), symmetric=True)
# The operator >> denotes matrix inequality.
constraints = [X >> 0]
constraints.append(cp.trace(new_A@X) == b[0])
prob = cp.Problem(cp.Minimize(cp.trace(C @ X)),
                  constraints)
prob.solve(solver = cp.SCS)

-0.037356172496962534

In [42]:
end_value = X.value
[(np.trace(A[i]@end_value) - b[i])**2 for  i in range(1, p)]

[0.002687916913259054, 0.07311652411082431, 2.9038175563066106]

In [43]:
new_A_2 = [new_A, A[2]]
X = cp.Variable((n,n), symmetric=True)
# The operator >> denotes matrix inequality.
constraints = [X >> 0]
constraints += [
    cp.trace(new_A_2[0]@X) == b[0],
    cp.trace(A[2]@X) == b[2]
]
prob = cp.Problem(cp.Minimize(cp.trace(C @ X)),
                  constraints)
prob.solve(solver = cp.SCS)

inf